In [ ]:
# https://www.properati.com.ar/data
! wget -O ar_properties.csv.gz -q https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz
! gzip -d -f ./ar_properties.csv.gz

In [33]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import datetime

# Lectura de datos

In [351]:
df_properties = pd.read_csv('./ar_properties.csv', index_col="id")
df_apred = pd.read_csv('./a_predecir.csv', index_col="id")

In [373]:
df = df_properties.copy()
df_pred = df_apred.copy()

In [31]:
df.head()

,start_date,end_date,created_on,l2,l3,l4,l5,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type,bath_rooms
id,,,,,,,,,,,,,,,,,,,,
LrDZTDP9vyylLQ3SmazY1A==,2020-08-22,2020-09-04,2020-08-22,Capital Federal,Boedo,NaN,NaN,0.0,0.0,0.0,520.0,520.0,70000.0,ARS,Mensual,Depósito con Oficina en Boedo,Depósito con amplias y cómodas oficinas y c...,Oficina,Alquiler,0
taiJHW87e1CmCPpuSgCbBA==,2020-08-22,2020-09-04,2020-08-22,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,NaN,0.0,0.0,2.0,95.0,0.0,35000.0,ARS,Mensual,Oficina - Paseo de la Bahia - Studios I,Excelente oportunidad! Oficina de 95 mts2 con ...,Oficina,Alquiler,1
RqacGyDGe/sKe4TYtuLLtg==,2020-08-22,2020-09-04,2020-08-22,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,NaN,0.0,0.0,2.0,90.0,0.0,27000.0,ARS,Mensual,Oficina - Paseo de la Bahia - Studios I,Excelente oportunidad! Oficina de 90 mts2 con ...,Oficina,Alquiler,1
hbIvfOetTyeiCcFPM4FATA==,2020-08-22,2020-09-04,2020-08-22,Capital Federal,Recoleta,NaN,NaN,0.0,0.0,3.0,265.0,265.0,65000.0,ARS,Mensual,Oficina comercial · 265m² · 2 Cochera,Gran Oportunidad! Unica piso de oficinas de 26...,Oficina,Alquiler,1
srXXAu7Fb8/Kng0/cxUe7g==,2020-08-22,2020-09-04,2020-08-22,Capital Federal,Balvanera,NaN,NaN,0.0,0.0,1.0,80.0,80.0,60000.0,ARS,Mensual,Oficina - Balvanera | Vista al pasaje Discépolo,"Edificio de oficinas desarrollado en Subsuelo,...",Oficina,Alquiler,1


In [393]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 1 to 1087
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   l3               1087 non-null   object 
 1   rooms            1083 non-null   float64
 2   bedrooms         958 non-null    float64
 3   bathrooms        1083 non-null   float64
 4   surface_total    1087 non-null   float64
 5   surface_covered  1087 non-null   float64
 6   price            0 non-null      float64
 7   property_type    1087 non-null   object 
dtypes: float64(6), object(2)
memory usage: 76.4+ KB


In [396]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 151918 entries, U3qdJMKXnOJm0Y1tWpnnfg== to RYlWKNYAOPbDbHGoYD/PAA==
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   l3               151342 non-null  object 
 1   rooms            151918 non-null  float64
 2   bedrooms         151918 non-null  float64
 3   bathrooms        151918 non-null  float64
 4   surface_total    151918 non-null  float64
 5   surface_covered  151918 non-null  float64
 6   price            151918 non-null  float64
 7   property_type    151918 non-null  object 
dtypes: float64(6), object(2)
memory usage: 10.4+ MB


# Análisis del conjunto de datos

In [256]:
#analizamos los valores de las distintas variables
df.ratio_supcubierta.value_counts()

0.000000    44411
1.000000    31513
0.909091     1398
0.900000     1173
0.888889     1043
            ...  
0.862595        1
0.404255        1
0.639241        1
0.387850        1
0.894988        1
Name: ratio_supcubierta, Length: 5340, dtype: int64

In [224]:
df.loc[df["price_period"] == "Mensual", "price"]

id
U3qdJMKXnOJm0Y1tWpnnfg==      85000.0
gsQB/JzLxaQdBLfNcm/DMw==     110000.0
SlPt6GJRjM+cO4rD3n3HFQ==     105000.0
ZaH+6DXJ4MLM6QqZXhgWiw==     150000.0
r7qrM5xaYo8QImSnqyKyYA==     136500.0
                              ...    
7aJWLQFXuwB0jjauufVCmg==    1500000.0
mzGbmoZd0oBbwlxDj06zWA==     770000.0
bBhDAJkRbJWukgHc0u//Zw==     318000.0
jaNlZUF/XN/I8PWilhABqw==    1350000.0
No3is+HhwUGlfgQfzsZbdg==    1400000.0
Name: price, Length: 67473, dtype: float64

In [61]:
df.start_date.max()

Timestamp('2021-06-17 00:00:00')

# Tratamiento de nulls y outliers

In [374]:
df.drop(columns = ["ad_type","l4","l5", "l6", "created_on","start_date", "end_date"], inplace = True )

In [375]:
df.dropna(subset=["price"], inplace = True)
df.dropna(subset=["currency"], inplace = True)

In [376]:
df = df.loc[df["l1"] == "Argentina"]

In [377]:
df = df.loc[df["l2"] == "Capital Federal"]

In [378]:
df.drop(columns = ["lat","lon", "l1","l2"], inplace = True )

In [379]:
df = df.loc[df["currency"] == "USD"]

In [380]:
df.drop(columns = ["currency"], inplace = True )

In [381]:
df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)

In [382]:
df = df.loc[df["property_type_check"] == 1]

In [383]:
df.drop(columns = "property_type_check", inplace = True)

In [384]:
df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)

In [ ]:
df_pred["rooms"].fillna(0, inplace = True)
df_pred["bedrooms"].fillna(0, inplace = True)
df_pred["bathrooms"].fillna(0, inplace = True)

In [385]:
df["rooms"].fillna(0, inplace = True)
df["bedrooms"].fillna(0, inplace = True)
df["bathrooms"].fillna(0, inplace = True)

In [386]:
df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]

/tmp/ipykernel_17351/4190027654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]


In [387]:
df["surface_total"].fillna(0, inplace = True)
df["surface_covered"].fillna(0, inplace = True)

In [388]:
df.drop(columns = ["price_period","operation_type"], inplace = True)

In [389]:
df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])

/tmp/ipykernel_17351/3611823614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])


In [390]:
df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])

/tmp/ipykernel_17351/3583135774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])


In [391]:
df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])

/tmp/ipykernel_17351/3604395483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])


In [ ]:
df.drop(columns = ["title","description"], inplace = True)

In [392]:
df_pred.drop(columns = ["ad_type","l4","l5", "l6", "created_on"], inplace = True )
df_pred.drop(columns = ["lat","lon", "l1","l2"], inplace = True )
df_pred.drop(columns = ["currency"], inplace = True )
df_pred.drop(columns = ["price_period"], inplace = True)
df_pred.drop(columns = "operation_type", inplace = True)
df_pred.drop(columns = ["start_date","end_date","services","additionals"], inplace = True)
df_pred.drop(columns = ["title","description","details"], inplace = True)

# Feature Engineering

In [397]:
df_total = pd.concat([df,df_pred], axis=0)

In [398]:
df_total["lnrooms"] = df_total.rooms.apply(lambda x: x if x == 0 else np.log(x))

In [400]:
df_total["lnbedrooms"] = df_total.bedrooms.apply(lambda x: x if x == 0 else np.log(x))

In [401]:
df_total = pd.get_dummies(df_total, columns = ["l3", "property_type"])

In [402]:
df = df_total[df_total.index.isin(df.index)]
df_pred = df_total[df_total.index.isin(df_pred.index)]

# Modelización

In [ ]:
X = df[df.columns.drop('price')]
y = df['price']

# for n_estimators in [100, 500, 1000]:     
#     for max_depth in [3, 7, 11]:     

## Tienen que usar RandomForestRegressor si o si o si. Pueden cambiar los parámetros
n_estimators = 500
max_depth = 4

reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)

scores = []
kf = sk.model_selection.KFold(n_splits=2, shuffle=True, random_state=42)
for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]

    # Entrenamos el modelo
    reg.fit(X_train, y_train)

    # Predecimos
    y_pred = reg.predict(X_test)

    # Medimos la performance de la predicción
    score = sk.metrics.mean_squared_error(y_test, y_pred)
    print(score)
    scores.append(score)

print(np.mean(scores), np.std(scores))

## Predicción para Kaggle


In [404]:
# quedate con las mismas columnas que ar_properties
X_pred = df_pred[X.columns]

In [415]:
# los mejores hiperparámetros encontrados antes
n_estimators = 5000
max_depth = 5

# Entrenamiento
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)

# Entrenamos el modelo
reg.fit(X_train, y_train)

# predicción
df_pred["price"] = reg.predict(X_pred)

/tmp/ipykernel_17351/3660555222.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["price"] = reg.predict(X_pred)


In [416]:
# Grabo la solución
df_pred[["price"]].to_csv("solucion.csv")